# Model Compression Toolkit (MCT) Wrapper API Comprehensive Quantization Comparison(tensorflow)

[Run this tutorial in Google Colab](https://colab.research.google.com/github/SonySemiconductorSolutions/mct-model-optimization/blob/main/tutorials/notebooks/mct_features_notebooks/keras/example_keras_mct_wrapper.ipynb)

## Overview 
This notebook provides a comprehensive demonstration of the MCT (Model Compression Toolkit) Wrapper API functionality, showcasing five different quantization methods on a MobileNetV2 model. The tutorial systematically compares the implementation, performance characteristics, and accuracy trade-offs of each quantization approach: PTQ (Post-Training Quantization), PTQ with Mixed Precision, GPTQ (Gradient-based PTQ), GPTQ with Mixed Precision, and LQ-PTQ (Low-bit Quantizer PTQ). Each method utilizes the unified MCTWrapper interface for consistent implementation and comparison.

## Summary
1. **Environment Setup**: Import required libraries and configure MCT with MobileNetV2 model
2. **Dataset Preparation**: Load and prepare ImageNet validation dataset with representative data generation
3. **PTQ Implementation**: Execute basic Post-Training Quantization with 8-bit precision and bias correction
4. **PTQ + Mixed Precision**: Apply intelligent bit-width allocation based on layer sensitivity analysis (75% compression ratio)
5. **GPTQ Implementation**: Perform gradient-based optimization with 5-epoch fine-tuning for enhanced accuracy
6. **GPTQ + Mixed Precision**: Combine gradient optimization with mixed precision for optimal accuracy-compression trade-off
7. **Performance Evaluation**: Comprehensive accuracy assessment and comparison across all quantization methods
8. **Results Analysis**: Compare model sizes, inference accuracy, and quantization trade-offs

## Setup

In [1]:
TF_VER = '2.14.0'
!pip install -q tensorflow~={TF_VER}

In [2]:
#import importlib
#if not importlib.util.find_spec('model_compression_toolkit'):
#    !pip install model_compression_toolkit

import sys
sys.path.append('/home/ubuntu/wrapper/sonyfork/mct-model-optimization')

import model_compression_toolkit as mct
from model_compression_toolkit.core import QuantizationErrorMethod

2025-11-06 07:02:27.950685: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-06 07:02:27.991442: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 07:02:28.190960: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-06 07:02:28.190993: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-06 07:02:28.192197: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
import keras
import tensorflow as tf

Load a pre-trained MobileNetV2 model from Keras, in 32-bits floating-point precision format.

In [4]:
from keras.applications.mobilenet_v2 import MobileNetV2

float_model = MobileNetV2()

2025-11-06 07:02:33.229062: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-06 07:02:33.233043: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Dataset preparation
Download ImageNet dataset with only the validation split.

**Note** that for demonstration purposes we use the validation set for the model quantization routines. Usually, a subset of the training dataset is used, but loading it is a heavy procedure that is unnecessary for the sake of this demonstration.

This step may take several minutes...

In [5]:
import os
 
if not os.path.isdir('imagenet'):
    !mkdir imagenet
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    
    !cd imagenet && tar -xzf ILSVRC2012_devkit_t12.tar.gz && \
     mkdir ILSVRC2012_img_val && tar -xf ILSVRC2012_img_val.tar -C ILSVRC2012_img_val

--2025-11-06 07:02:33--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
Resolving proxy.mei.co.jp (proxy.mei.co.jp)... 10.77.8.70
Connecting to proxy.mei.co.jp (proxy.mei.co.jp)|10.77.8.70|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 2568145 (2.4M) [application/x-gzip]
Saving to: ‘imagenet/ILSVRC2012_devkit_t12.tar.gz’

ILSVRC2012_devkit_t 100%[===================>]   2.45M  1.99MB/s    in 1.2s    

2025-11-06 07:02:35 (1.99 MB/s) - ‘imagenet/ILSVRC2012_devkit_t12.tar.gz’ saved [2568145/2568145]

--2025-11-06 07:02:35--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving proxy.mei.co.jp (proxy.mei.co.jp)... 10.77.8.70
Connecting to proxy.mei.co.jp (proxy.mei.co.jp)|10.77.8.70|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘imagenet/ILSVRC2012_img_val.tar’

ILSVRC2012_img_val. 100%[===================>]   6.28G  10.7MB/s    in 33m 51s 

2

The following code organizes the extracted data into separate folders for each label, making it compatible with Keras dataset loaders.

In [6]:
from pathlib import Path
import shutil

root = Path('./imagenet')
imgs_dir = root / 'ILSVRC2012_img_val'
target_dir = root /'val'

def extract_labels():
    !pip install -q scipy
    import scipy
    mat = scipy.io.loadmat(root / 'ILSVRC2012_devkit_t12/data/meta.mat', squeeze_me=True)
    cls_to_nid = {s[0]: s[1] for i, s in enumerate(mat['synsets']) if s[4] == 0} 
    with open(root / 'ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt', 'r') as f:
        return [cls_to_nid[int(cls)] for cls in f.readlines()]

if not target_dir.exists():
    labels = extract_labels()
    for lbl in set(labels):
        os.makedirs(target_dir / lbl)
    
    for img_file, lbl in zip(sorted(os.listdir(imgs_dir)), labels):
        shutil.move(imgs_dir / img_file, target_dir / lbl)

These functions generate a `tf.data.Dataset` from image files in a directory.

In [7]:
def imagenet_preprocess_input(images, labels):
    return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels

def get_dataset(batch_size, shuffle):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory='./imagenet/val',
        batch_size=batch_size,
        image_size=[224, 224],
        shuffle=shuffle,
        crop_to_aspect_ratio=True,
        interpolation='bilinear')
    dataset = dataset.map(lambda x, y: (imagenet_preprocess_input(x, y)), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

## Representative Dataset
For quantization with MCT, we need to define a representative dataset required by the PTQ algorithm. This dataset is a generator that returns a list of images:

In [8]:
batch_size = 16
n_iter = 5

dataset = get_dataset(batch_size, shuffle=True)

def representative_dataset_gen():
    for _ in range(n_iter):
        yield [dataset.take(1).get_single_element()[0].numpy()]

Found 50000 files belonging to 1000 classes.


## Model Post-Training quantization using MCTWrapper

In [9]:
# Decorator to provide consistent logging and error handling for quantization functions
def decorator(func):
    """
    Wrapper decorator that provides standardized execution logging and error handling.
    
    This decorator enhances quantization functions by:
    - Providing clear start/end execution markers for debugging
    - Handling success/failure status from quantization operations
    - Implementing fail-fast behavior on quantization errors
    - Ensuring consistent logging format across all quantization methods
    
    Usage:
        @decorator
        def quantization_function(model):
            # quantization implementation
            return flag, quantized_model
    
    Args:
        func: Function to be decorated (typically a quantization function)
    
    Returns:
        Wrapped function with enhanced logging and error handling capabilities
    """
    def wrapper(*args, **kwargs):
        # Log function execution start with clear delimiter
        print(f"----------------- {func.__name__} Start ---------------")
        
        # Execute the quantization function and capture return values
        # Expected return format: (success_flag, quantized_model)
        flag, result = func(*args, **kwargs)
        
        # Log function execution completion
        print(f"----------------- {func.__name__} End -----------------")
        
        # Implement fail-fast behavior: exit immediately on quantization failure
        # This ensures early detection of quantization issues
        if not flag:
            exit()
        
        # Return original function results if successful
        return flag, result
    
    return wrapper

Run PTQ (Post-Training Quantization) with Keras

In [10]:
@decorator
def PTQ_Keras(float_model):
    """
    Perform Post-Training Quantization (PTQ) using MCT on Keras model.
    
    PTQ is a quantization method that:
    - Does not require model retraining
    - Uses representative data for calibration
    - Provides good accuracy with minimal computational overhead
    
    Args:
        float_model: Original floating-point Keras model
    
    Returns:
        tuple: (success_flag, quantized_model)
    """
    # Configuration for basic PTQ quantization
    method = 'PTQ'                    # Post-Training Quantization method
    framework = 'tensorflow'          # Target framework (Keras/TensorFlow)
    use_internal_tpc = True                # Use MCT's built-in Target Platform Capabilities
    use_mixed_precision = False                  # Disable mixed-precision quantization

    # Parameter configuration for PTQ
    param_items = [
        ['tpc_version', '1.0', 'The version of the TPC to use.'],
        
        # Quantization configuration parameters
        ['activation_error_method', QuantizationErrorMethod.MSE, 'Error metric for activation quantization'],
        ['weights_bias_correction', True, 'Enable bias correction for weights'],
        ['z_threshold', float('inf'), 'Threshold for zero-point quantization'],
        ['linear_collapsing', True, 'Enable linear layer collapsing optimization'],
        ['residual_collapsing', True, 'Enable residual connection collapsing'],
        
        # Output configuration
        ['save_model_path', './qmodel_PTQ_Keras.tflite', 'Path to save the quantized model']
    ]

    # Execute quantization using MCTWrapper
    wrapper = mct.wrapper.mct_wrapper.MCTWrapper()
    flag, quantized_model = wrapper.quantize_and_export(
        float_model, method, framework, use_internal_tpc, use_mixed_precision, 
        representative_dataset_gen, param_items)
    return flag, quantized_model

Run PTQ + Mixed Precision Quantization with Keras

In [11]:
@decorator
def PTQ_Keras_mixed_precision(float_model):
    """
    Perform Post-Training Quantization with Mixed Precision (PTQ + mixed_precision) on Keras model.
    
    Mixed Precision Quantization:
    - Uses different bit-widths for different layers
    - Optimizes model size while maintaining accuracy
    - Automatically selects optimal precision for each layer
    - Uses resource constraints to guide precision allocation
    
    Args:
        float_model: Original floating-point Keras model
    
    Returns:
        tuple: (success_flag, quantized_model)
    """
    # Configuration for PTQ with mixed precision
    method = 'PTQ'                    # Post-Training Quantization method
    framework = 'tensorflow'          # Target framework (Keras/TensorFlow)
    use_internal_tpc = True                # Use MCT's built-in Target Platform Capabilities
    use_mixed_precision = True                   # Enable mixed-precision quantization

    # Parameter configuration for PTQ with Mixed Precision
    param_items = [
        ['tpc_version', '1.0', 'The version of the TPC to use.'],
        
        # Mixed precision configuration
        ['num_of_images', 5, 'Number of images for mixed precision analysis'],
        ['use_hessian_based_scores', False, 'Use Hessian-based sensitivity scores for layer importance'],
        
        # Resource constraint configuration
        ['weights_compression_ratio', 0.75, 'Target compression ratio for model weights (75% of original size)'],
        
        # Output configuration
        ['save_model_path', './qmodel_PTQ_Keras_mixed_precision.tflite', 'Path to save the mixed precision quantized model']
    ]

    # Execute mixed precision quantization using MCTWrapper
    wrapper = mct.wrapper.mct_wrapper.MCTWrapper()
    flag, quantized_model = wrapper.quantize_and_export(
        float_model, method, framework, use_internal_tpc, use_mixed_precision, 
        representative_dataset_gen, param_items)
    return flag, quantized_model

Run GPTQ (Gradient-based PTQ) with Keras

In [12]:
@decorator
def GPTQ_Keras(float_model):
    """
    Perform Gradient-based Post-Training Quantization (GPTQ) on Keras model.
    
    GPTQ is an advanced quantization method that:
    - Uses gradient information to optimize quantization parameters
    - Fine-tunes the model during quantization process
    - Generally provides better accuracy than standard PTQ
    - Requires slightly more computational resources than PTQ
    
    Args:
        float_model: Original floating-point Keras model
    
    Returns:
        tuple: (success_flag, quantized_model)
    """
    # Configuration for GPTQ quantization
    method = 'GPTQ'                   # Gradient-based Post-Training Quantization
    framework = 'tensorflow'          # Target framework (Keras/TensorFlow)
    use_internal_tpc = True                # Use external EdgeMDT Target Platform Capabilities
    use_mixed_precision = False                  # Disable mixed-precision quantization

    # Parameter configuration for GPTQ
    param_items = [
        # Platform configuration
        ['target_platform_version', 'v1', 'Target platform capabilities version'],
        
        # GPTQ-specific training parameters
        ['n_epochs', 5, 'Number of epochs for gradient-based fine-tuning'],
        ['optimizer', None, 'Optimizer for fine-tuning (None = use default)'],
        
        # Output configuration
        ['save_model_path', './qmodel_GPTQ_Keras.tflite', 'Path to save the GPTQ quantized model']
    ]

    # Execute GPTQ quantization using MCTWrapper
    wrapper = mct.wrapper.mct_wrapper.MCTWrapper()
    flag, quantized_model = wrapper.quantize_and_export(
        float_model, method, framework, use_internal_tpc, use_mixed_precision, 
        representative_dataset_gen, param_items)
    return flag, quantized_model

Run GPTQ + Mixed Precision Quantization with Keras

In [13]:
@decorator
def GPTQ_Keras_mixed_precision(float_model):
    """
    Perform Gradient-based Post-Training Quantization with Mixed Precision (GPTQ + mixed_precision).
    
    This combines the benefits of both techniques:
    - GPTQ: Gradient-based optimization for better quantization accuracy
    - Mixed Precision: Optimal bit-width allocation for size/accuracy trade-off
    
    This is the most advanced quantization method available, providing:
    - Best possible accuracy preservation
    - Optimal model size reduction
    - Automatic precision selection per layer
    
    Args:
        float_model: Original floating-point Keras model
    
    Returns:
        tuple: (success_flag, quantized_model)
    """
    # Configuration for GPTQ with mixed precision
    method = 'GPTQ'                   # Gradient-based Post-Training Quantization
    framework = 'tensorflow'          # Target framework (Keras/TensorFlow)
    use_internal_tpc = True                # Use external EdgeMDT Target Platform Capabilities
    use_mixed_precision = True                   # Enable mixed-precision quantization

    # Parameter configuration for GPTQ with Mixed Precision
    param_items = [
        # Platform configuration
        ['target_platform_version', 'v1', 'Target platform capabilities version'],
        
        # GPTQ-specific training parameters
        ['n_epochs', 5, 'Number of epochs for gradient-based fine-tuning'],
        ['optimizer', None, 'Optimizer for fine-tuning (None = use default)'],
        
        # Mixed precision configuration
        ['num_of_images', 5, 'Number of images for mixed precision sensitivity analysis'],
        ['use_hessian_based_scores', False, 'Use Hessian-based scores for layer importance ranking'],
        
        # Resource constraint configuration
        ['weights_compression_ratio', 0.75, 'Target compression ratio for model weights (75% reduction)'],
        
        # Output configuration
        ['save_model_path', './qmodel_GPTQ_Keras_mixed_precision.tflite', 'Path to save the GPTQ+mixed_precision quantized model']
    ]

    # Execute advanced GPTQ+mixed_precision quantization using MCTWrapper
    wrapper = mct.wrapper.mct_wrapper.MCTWrapper()
    flag, quantized_model = wrapper.quantize_and_export(
        float_model, method, framework, use_internal_tpc, use_mixed_precision, 
        representative_dataset_gen, param_items)
    return flag, quantized_model

### Run model Post-Training Quantization
Lastly, we quantize our model using MCTWrapper API.

In [14]:
# Load pre-trained MobileNetV2 model as the base model for quantization experiments
# This model serves as the reference floating-point model for all quantization methods
float_model = MobileNetV2()

# Execute comprehensive quantization method comparison using MCT Wrapper functionality
# Each method represents different trade-offs between accuracy, model size, and computation time
print("Starting quantization experiments with different methods...")

Starting quantization experiments with different methods...


In [15]:
# Method 1: Basic Post-Training Quantization (PTQ)
# - Standard 8-bit quantization without advanced optimization techniquesed
flag, quantized_model = PTQ_Keras(float_model)

----------------- PTQ_Keras Start ---------------


Statistics Collection: 5it [00:13,  2.62s/it]
Calculating quantization parameters: 100%|████████████████████████████████| 105/105 [00:08<00:00, 11.86it/s]


INFO:tensorflow:Assets written to: /tmp/tmp6akw4o6x/assets


INFO:tensorflow:Assets written to: /tmp/tmp6akw4o6x/assets


----------------- PTQ_Keras End -----------------


2025-11-06 07:37:07.968689: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-06 07:37:07.968728: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-06 07:37:07.969430: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp6akw4o6x
2025-11-06 07:37:07.976394: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-06 07:37:07.976406: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp6akw4o6x
2025-11-06 07:37:07.991779: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2025-11-06 07:37:07.994463: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-06 07:37:08.084052: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp6akw4o6x
2025-11

In [16]:
# Method 2: PTQ with Mixed Precision Quantization
# - Uses different bit-widths for different layers based on sensitivity analysis
flag, quantized_model2 = PTQ_Keras_mixed_precision(float_model)

----------------- PTQ_Keras_mixed_precision Start ---------------


Statistics Collection: 5it [00:13,  2.62s/it]
Calculating quantization parameters: 100%|████████████████████████████████| 105/105 [00:26<00:00,  4.03it/s]
53it [00:22,  2.39it/s]


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ubuntu/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/bca1c515b45e4c878019f8094542eba7-pulp.mps -sec 60 -timeMode elapsed -branch -printingOptions all -solution /tmp/bca1c515b45e4c878019f8094542eba7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 112 COLUMNS
At line 961 RHS
At line 1069 BOUNDS
At line 1229 ENDATA
Problem MODEL has 107 rows, 212 columns and 477 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.0457804 - 0.00 seconds
Cgl0004I processed model has 54 rows, 159 columns (159 integer (159 of which binary)) and 265 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.589583
Cbc0038I Pass   1: suminf.    0.14150 (2) obj. 0.0621947 iterations 2
Cbc0038I Solution found of 0.300313
Cbc0038I Befor

INFO:tensorflow:Assets written to: /tmp/tmpymthg94f/assets


----------------- PTQ_Keras_mixed_precision End -----------------


2025-11-06 07:38:17.750681: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-06 07:38:17.750717: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-06 07:38:17.750855: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpymthg94f
2025-11-06 07:38:17.756046: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-06 07:38:17.756062: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpymthg94f
2025-11-06 07:38:17.771202: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-06 07:38:17.863602: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpymthg94f
2025-11-06 07:38:17.899600: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 148745 

In [17]:
# Method 3: Gradient-based Post-Training Quantization (GPTQ)
# - Uses gradient information to fine-tune quantization parameters during conversion
flag, quantized_model3 = GPTQ_Keras(float_model)

----------------- GPTQ_Keras Start ---------------


Statistics Collection: 5it [00:13,  2.75s/it]
Calculating quantization parameters: 100%|████████████████████████████████| 105/105 [00:09<00:00, 10.81it/s]
Estimating representative dataset size: 5it [00:00, 31.81it/s]
Running GPTQ optimization: 100%|██████████████████████████████████████████████| 5/5 [00:21<00:00,  4.21s/it]


INFO:tensorflow:Assets written to: /tmp/tmpc4davibj/assets


INFO:tensorflow:Assets written to: /tmp/tmpc4davibj/assets


----------------- GPTQ_Keras End -----------------


2025-11-06 08:31:19.418708: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-06 08:31:19.418748: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-06 08:31:19.418879: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpc4davibj
2025-11-06 08:31:19.424929: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-06 08:31:19.424948: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpc4davibj
2025-11-06 08:31:19.441242: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-06 08:31:19.538790: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpc4davibj
2025-11-06 08:31:19.578268: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 159390 

In [18]:
# Method 4: GPTQ with Mixed Precision Quantization
# - Combines gradient-based optimization with mixed precision techniques
flag, quantized_model4 = GPTQ_Keras_mixed_precision(float_model)

----------------- GPTQ_Keras_mixed_precision Start ---------------


Statistics Collection: 5it [00:13,  2.76s/it]
Calculating quantization parameters: 100%|████████████████████████████████| 105/105 [00:28<00:00,  3.70it/s]
53it [00:22,  2.34it/s]


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ubuntu/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/ce5f1a4cdda34ce2ad0392304eca0326-pulp.mps -sec 60 -timeMode elapsed -branch -printingOptions all -solution /tmp/ce5f1a4cdda34ce2ad0392304eca0326-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 112 COLUMNS
At line 961 RHS
At line 1069 BOUNDS
At line 1229 ENDATA
Problem MODEL has 107 rows, 212 columns and 477 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.0603818 - 0.00 seconds
Cgl0004I processed model has 54 rows, 159 columns (159 integer (159 of which binary)) and 265 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.589583
Cbc0038I Pass   1: suminf.    0.14150 (2) obj. 0.0800306 iterations 2
Cbc0038I Solution found of 0.372231
Cbc0038I Befor

Running GPTQ optimization: 100%|██████████████████████████████████████████████| 5/5 [00:18<00:00,  3.61s/it]


INFO:tensorflow:Assets written to: /tmp/tmpcjk7c1iv/assets


INFO:tensorflow:Assets written to: /tmp/tmpcjk7c1iv/assets


----------------- GPTQ_Keras_mixed_precision End -----------------


2025-11-06 09:26:25.535563: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-06 09:26:25.535641: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-06 09:26:25.536032: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpcjk7c1iv
2025-11-06 09:26:25.541989: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-06 09:26:25.542008: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpcjk7c1iv
2025-11-06 09:26:25.558172: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-06 09:26:25.656187: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpcjk7c1iv
2025-11-06 09:26:25.691559: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 155493 

In [19]:
print("All quantization methods completed successfully!")

All quantization methods completed successfully!


## Models evaluation
In order to evaluate our models, we first need to load the validation dataset. As before, please ensure that the dataset path has been set correctly.

In [21]:
# Model Evaluation and Accuracy Comparison
print("Starting model evaluation phase...")

# Prepare validation dataset for accuracy assessment
val_dataset = get_dataset(batch_size=50, shuffle=False)

Starting model evaluation phase...
Found 50000 files belonging to 1000 classes.


In [22]:
# Evaluate original floating-point model accuracy
print("\n=== Original Model Evaluation ===")
float_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
float_accuracy = float_model.evaluate(val_dataset)
print(f"Float model's Top 1 accuracy on the Imagenet validation set: {(float_accuracy[1] * 100):.2f}%")


=== Original Model Evaluation ===
1000/1000 [==============================] - 167s 166ms/step - loss: 1.2200 - accuracy: 0.7185
Float model's Top 1 accuracy on the Imagenet validation set: 71.85%


In [23]:
# Evaluate PTQ quantized model accuracy
print("\n=== PTQ Model Evaluation ===")
quantized_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
quantized_accuracy = quantized_model.evaluate(val_dataset)
print(f"PTQ_Keras Quantized model's Top 1 accuracy on the Imagenet validation set: {(quantized_accuracy[1] * 100):.2f}%")


=== PTQ Model Evaluation ===
1000/1000 [==============================] - 156s 156ms/step - loss: 1.3538 - accuracy: 0.7164
PTQ_Keras Quantized model's Top 1 accuracy on the Imagenet validation set: 71.64%


In [24]:
# Evaluate PTQ + Mixed Precision model accuracy
print("\n=== PTQ + Mixed Precision Model Evaluation ===")
quantized_model2.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
quantized_accuracy = quantized_model2.evaluate(val_dataset)
print(f"PTQ_Keras_mixed_precision Quantized model's Top 1 accuracy on the Imagenet validation set: {(quantized_accuracy[1] * 100):.2f}%")


=== PTQ + Mixed Precision Model Evaluation ===
1000/1000 [==============================] - 158s 157ms/step - loss: 1.3545 - accuracy: 0.7159
PTQ_Keras_mixed_precision Quantized model's Top 1 accuracy on the Imagenet validation set: 71.59%


In [25]:
# Evaluate GPTQ quantized model accuracy
print("\n=== GPTQ Model Evaluation ===")
quantized_model3.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
quantized_accuracy = quantized_model3.evaluate(val_dataset)
print(f"GPTQ_Keras Quantized model's Top 1 accuracy on the Imagenet validation set: {(quantized_accuracy[1] * 100):.2f}%")


=== GPTQ Model Evaluation ===
1000/1000 [==============================] - 158s 156ms/step - loss: 1.3649 - accuracy: 0.7137
GPTQ_Keras Quantized model's Top 1 accuracy on the Imagenet validation set: 71.37%


In [26]:
# Evaluate GPTQ + Mixed Precision model accuracy
print("\n=== GPTQ + Mixed Precision Model Evaluation ===")
quantized_model4.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
quantized_accuracy = quantized_model4.evaluate(val_dataset)
print(f"GPTQ_Keras_mixed_precision Quantized model's Top 1 accuracy on the Imagenet validation set: {(quantized_accuracy[1] * 100):.2f}%")


=== GPTQ + Mixed Precision Model Evaluation ===
1000/1000 [==============================] - 158s 157ms/step - loss: 1.3859 - accuracy: 0.7107
GPTQ_Keras_mixed_precision Quantized model's Top 1 accuracy on the Imagenet validation set: 71.07%


In [ ]:
print("Fisish")

## Conclusion

In this tutorial, we demonstrated how to quantize a pre-trained model using MCTWrapper with a few lines of code.

MCT can deliver competitive results across a wide range of tasks and network architectures. For more details, [check out the paper:](https://arxiv.org/abs/2109.09113).

## Copyrights

Copyright 2025 Sony Semiconductor Solutions, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
